In [3]:
import pandas as pd
import datetime as dt

# ---------------------------------------------------------
# GLOBAL AYARLAR (CONFIGURATION)
# ---------------------------------------------------------

# Float değerlerin virgülden sonra 2 basamak gösterilmesi (Okunabilirlik ayarı)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Tüm sütunların konsolda görüntülenebilmesi için limitin kaldırılması
pd.set_option('display.max_columns', None)

# Uyarı mesajlarını yönetmek için (Opsiyonel, temiz çıktı için)
import warnings
warnings.filterwarnings("ignore")

In [4]:
# ---------------------------------------------------------
# DATA INGESTION (VERİ YÜKLEME)
# ---------------------------------------------------------

# Veri setini zip'ten çıkarma
!unzip -q "online+retail+ii.zip"

# Veri setini yükleme
print("[INFO] Veri seti yükleniyor...")
df_raw = pd.read_excel("online_retail_II.xlsx", sheet_name="Year 2010-2011")

# Orijinal veriyi korumak için kopya oluşturma
df = df_raw.copy()

# Veri seti boyutunu ve ilk kayıtları gösterme
print(f"[INFO] Veri seti yüklendi. Boyut: {df.shape}")
print(df.head())

[INFO] Veri seti yükleniyor...
[INFO] Veri seti yüklendi. Boyut: (541910, 8)
  Invoice StockCode                          Description  Quantity  \
0  536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1  536365     71053                  WHITE METAL LANTERN         6   
2  536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3  536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4  536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  Price  Customer ID         Country  
0 2010-12-01 08:26:00   2.55     17850.00  United Kingdom  
1 2010-12-01 08:26:00   3.39     17850.00  United Kingdom  
2 2010-12-01 08:26:00   2.75     17850.00  United Kingdom  
3 2010-12-01 08:26:00   3.39     17850.00  United Kingdom  
4 2010-12-01 08:26:00   3.39     17850.00  United Kingdom  


In [5]:
# ---------------------------------------------------------
# DATA PREPROCESSING & RFM CALCULATIONS
# ---------------------------------------------------------

def prepare_data(dataframe):
    """
    Veri temizliği, tip dönüşümleri ve yeni değişken üretimini gerçekleştirir.

    Args:
        dataframe (pd.DataFrame): Ham veri seti.

    Returns:
        pd.DataFrame: Temizlenmiş ve RFM analizi için hazırlanmış veri seti.
    """
    df = dataframe.copy()

    # 1. Eksik Gözlemlerin Giderilmesi
    # Customer ID'si olmayan işlemler analiz edilemez.
    df.dropna(subset=['Customer ID'], inplace=True)

    # 2. İade İşlemlerinin Filtrelenmesi
    # 'C' harfi içeren faturalar iadedir.
    df = df[~df["Invoice"].astype(str).str.contains("C", na=False)]

    # 3. Aykırı/Hatalı Değer Temizliği
    # Miktar ve Fiyat 0'dan büyük olmalıdır.
    df = df[(df['Quantity'] > 0) & (df['Price'] > 0)]

    # 4. Feature Engineering (Yeni Değişken Üretimi)
    # Toplam Tutar (Monetary metriği için temel)
    df["TotalPrice"] = df["Quantity"] * df["Price"]

    # 5. Tip Dönüşümleri
    df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])
    df["Customer ID"] = df["Customer ID"].astype(int)

    return df

def calculate_rfm_metrics(dataframe):
    """
    Hazırlanmış veriden Recency, Frequency ve Monetary metriklerini hesaplar.
    """
    # Analiz tarihi, veri setindeki son işlem tarihinden 2 gün sonrası olarak belirlenir.
    analysis_date = dataframe["InvoiceDate"].max() + dt.timedelta(days=2)

    # RFM Metriklerinin Hesaplanması
    rfm = dataframe.groupby('Customer ID').agg({
        'InvoiceDate': lambda date: (analysis_date - date.max()).days, # Recency
        'Invoice': 'nunique',                                          # Frequency
        'TotalPrice': 'sum'                                            # Monetary
    })

    # Sütunların yeniden isimlendirilmesi
    rfm.columns = ['Recency', 'Frequency', 'Monetary']

    # Monetary değeri 0 olan (varsa) kayıtların temizlenmesi
    rfm = rfm[rfm["Monetary"] > 0]

    return rfm

def generate_rfm_scores(rfm_df):
    """
    RFM metriklerini istatistiksel olarak puanlar (qcut) ve segmentlere ayırır.

    Segmentasyon Haritası:
    - Hibernating, At_Risk, Cant_Loose, About_to_Sleep, Need_Attention,
    - Loyal_Customers, Promising, New_Customers, Potential_Loyalists, Champions
    """
    # 1. Skorlama (1-5 Skalası)
    # Recency: Düşük değer daha iyidir (Ters orantı -> labels=[5,4,3,2,1])
    rfm_df["Recency_Score"] = pd.qcut(rfm_df['Recency'], 5, labels=[5, 4, 3, 2, 1])

    # Frequency & Monetary: Yüksek değer daha iyidir (Doğru orantı -> labels=[1,2,3,4,5])
    # Not: Frequency'de çok fazla tekrar eden değer olabileceği için rank(method="first") kullanılır.
    rfm_df["Frequency_Score"] = pd.qcut(rfm_df['Frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
    rfm_df["Monetary_Score"] = pd.qcut(rfm_df['Monetary'], 5, labels=[1, 2, 3, 4, 5])

    # 2. RF Skorunun Oluşturulması (String concatenation)
    rfm_df["RF_SCORE"] = (rfm_df['Recency_Score'].astype(str) +
                          rfm_df['Frequency_Score'].astype(str))

    # 3. Segment İsimlendirmesi (Regex Mapping)
    seg_map = {
        r'[1-2][1-2]': 'Hibernating',
        r'[1-2][3-4]': 'At_Risk',
        r'[1-2]5': 'Cant_Loose',
        r'3[1-2]': 'About_to_Sleep',
        r'33': 'Need_Attention',
        r'[3-4][4-5]': 'Loyal_Customers',
        r'41': 'Promising',
        r'51': 'New_Customers',
        r'[4-5][2-3]': 'Potential_Loyalists',
        r'5[4-5]': 'Champions'
    }

    rfm_df['Segment'] = rfm_df['RF_SCORE'].replace(seg_map, regex=True)

    return rfm_df

In [6]:
# ---------------------------------------------------------
# EXECUTION & REPORTING (ÇALIŞTIRMA VE RAPORLAMA)
# ---------------------------------------------------------

print("[INFO] Veri ön işleme başlıyor...")
processed_df = prepare_data(df)
print(f"[INFO] Ön işleme tamamlandı. Kalan Satır Sayısı: {processed_df.shape[0]}")

print("[INFO] RFM metrikleri hesaplanıyor...")
rfm_metrics = calculate_rfm_metrics(processed_df)

print("[INFO] Skorlama ve segmentasyon yapılıyor...")
final_rfm = generate_rfm_scores(rfm_metrics)

# SONUÇLARIN GÖSTERİMİ
print("\n" + "#"*40)
print("ANALİZ RAPORU")
print("#"*40 + "\n")

print("--- Segment Bazlı Müşteri Dağılımı ---")
print(final_rfm["Segment"].value_counts())

print("\n--- Segmentlerin Metrik Ortalamaları ---")
print(final_rfm.groupby("Segment")[["Recency", "Frequency", "Monetary"]].mean())

print("\n--- İlk 5 Müşteri Örneği ---")
print(final_rfm.head())

[INFO] Veri ön işleme başlıyor...
[INFO] Ön işleme tamamlandı. Kalan Satır Sayısı: 397885
[INFO] RFM metrikleri hesaplanıyor...
[INFO] Skorlama ve segmentasyon yapılıyor...

########################################
ANALİZ RAPORU
########################################

--- Segment Bazlı Müşteri Dağılımı ---
Segment
Hibernating            1065
Loyal_Customers         827
Champions               633
At_Risk                 580
Potential_Loyalists     492
About_to_Sleep          351
Need_Attention          186
Promising                99
Cant_Loose               63
New_Customers            42
Name: count, dtype: int64

--- Segmentlerin Metrik Ortalamaları ---
                     Recency  Frequency  Monetary
Segment                                          
About_to_Sleep         54.50       1.16    461.06
At_Risk               156.06       2.86   1076.51
Cant_Loose            133.43       8.38   2796.16
Champions               6.88      12.41   6857.96
Hibernating           218.90      

In [8]:
# ---------------------------------------------------------
# DATA EXPORT (RAPORLAMA VE DIŞA AKTARIM)
# ---------------------------------------------------------

def export_results(dataframe, filename="rfm_analysis_results"):
    """
    Analiz sonuçlarını kalıcı depolama veya raporlama araçları için dışa aktarır.

    Args:
        dataframe (pd.DataFrame): Segmentasyon sonuçlarını içeren veri seti.
        filename (str): Çıktı dosyasının baz ismi.
    """
    try:
        # Versiyon kontrolü ve arşivleme kolaylığı için zaman damgası (timestamp) eklenmesi
        date_str = dt.datetime.now().strftime("%Y-%m-%d")
        final_filename = f"{filename}_{date_str}.csv"

        # Sonuçların kaydedilmesi (Customer ID index'te olduğu için index=True tutulur)
        dataframe.to_csv(final_filename)

        print(f"[INFO] Dosya başarıyla export edildi: {final_filename}")

    except Exception as e:
        print(f"[ERROR] Dosya kaydedilirken hata oluştu: {e}")

# Fonksiyonu çalıştırma
export_results(final_rfm)

[INFO] Dosya başarıyla export edildi: rfm_analysis_results_2025-12-22.csv


In [11]:
# ---------------------------------------------------------
# DATA VISUALIZATION (VERİ GÖRSELLEŞTİRME & RAPORLAMA)
# ---------------------------------------------------------
import plotly.express as px

def visualize_rfm_segments(dataframe):
    """
    Segmentasyon sonuçlarını etkileşimli grafiklerle görselleştirir.

    Kullanılan Grafikler:
    1. Treemap: Segmentlerin oransal büyüklüklerini gösterir.
    2. Scatter Plot: Müşteri dağılımını Recency/Frequency ekseninde analiz eder.
    """

    # Görselleştirme kütüphanesi (Plotly) veriyi sütun bazlı işler.
    # 'Customer ID' indeks yapısında olduğu için, erişilebilir olması adına sütuna dönüştürülür.
    df_vis = dataframe.reset_index()

    # -----------------------------------------------------
    # 1. TREEMAP: Segment Dağılım Analizi
    # Amaç: Hangi segmentin domine ettiğini makro seviyede görmek.
    # -----------------------------------------------------
    segment_counts = df_vis['Segment'].value_counts().reset_index()
    segment_counts.columns = ['Segment', 'Count']

    fig_treemap = px.treemap(segment_counts,
                             path=['Segment'],
                             values='Count',
                             title='Müşteri Segment Dağılımı (Treemap)',
                             color='Count',
                             color_continuous_scale='RdBu')
    fig_treemap.show()

    # -----------------------------------------------------
    # 2. SCATTER PLOT: Müşteri Değer Matrisi
    # Amaç: Recency ve Frequency arasındaki korelasyonu ve parasal değeri görmek.
    # -----------------------------------------------------

    fig_scatter = px.scatter(df_vis,
                             x="Recency",
                             y="Frequency",
                             color="Segment",
                             size="Monetary",            # Baloncuk hacmi ciro katkısını temsil eder
                             hover_data=["Customer ID"], # Drill-down analiz için müşteri ID detayı
                             log_x=True,                 # Dağılımın daha net görülmesi için logaritmik eksen
                             log_y=True,
                             title="RFM Matrisi: Recency vs Frequency (Log Scale)",
                             template="plotly_white")

    fig_scatter.show()

print("[INFO] Görselleştirme modülü çalıştırılıyor...")
visualize_rfm_segments(final_rfm)

[INFO] Görselleştirme modülü çalıştırılıyor...


In [13]:
# ---------------------------------------------------------
# BUSINESS ACTION PLAN (İŞ AKSİYON ÖNERİLERİ)
# ---------------------------------------------------------

def recommend_actions(dataframe):
    """
    Segmentasyon sonuçlarını işlenebilir stratejilere dönüştürür.
    En yoğun 3 segment için aksiyon önerileri sunar.
    """

    # Hedef Kitle Belirleme: En kalabalık 3 segment
    top_segments = dataframe['Segment'].value_counts().head(3).index.tolist()

    print("\n" + "="*65)
    print("STRATEJİK AKSİYON PLANI (YÖNETİCİ ÖZETİ)")
    print("="*65)

    # Strateji Kütüphanesi
    strategies = {
        'Champions':
            "✨ [SEGMENT: ŞAMPİYONLAR]\n"
            "   Durum: En son, en sık ve en çok alışveriş yapanlar.\n"
            "   Action: 'Marka Elçisi' programına dahil edin.\n"
            "   Taktik: Yeni ürünleri ilk onlara sunun (Erken Erişim). Özel %5 teşekkür indirimi tanımlayın.\n",

        'Loyal_Customers':
            "🔄 [SEGMENT: SADIK MÜŞTERİLER]\n"
            "   Durum: Düzenli geliyorlar ancak harcamaları arttırılabilir.\n"
            "   Action: Sepet Ortalamasını Yükseltme (Upsell/Cross-sell).\n"
            "   Taktik: İlgi alanlarına göre yan ürün önerileri yapın. Kargo bedava limitini hatırlatın.\n",

        'Hibernating':
            "💤 [SEGMENT: UYKUDAKİLER]\n"
            "   Durum: Uzun süredir gelmediler ve alışveriş sıklıkları düşüktü.\n"
            "   Action: Maliyet Odaklı Geri Kazanım.\n"
            "   Taktik: Sadece e-posta kanalıyla, 'Seni Özledik' temalı agresif bir indirim kuponu iletin.\n",

        'At_Risk':
            "⚠️ [SEGMENT: RİSK GRUBU]\n"
            "   Durum: Eskiden iyi müşterilerdi ancak yakın zamanda kaybolmak üzereler.\n"
            "   Action: Acil Müdahale (Churn Prevention).\n"
            "   Taktik: Kişiselleştirilmiş bir bildirim gönderin. 'Bizi neden bıraktınız?' anketi ile hediye çeki sunun.\n",

        'Potential_Loyalists':
             "📈 [SEGMENT: POTANSİYEL SADIKLAR]\n"
             "   Durum: Yeni yeni alışkanlık kazanıyorlar.\n"
             "   Action: Sadakat Programına Katılım.\n"
             "   Taktik: İlk 3 alışverişe özel puan sistemi sunarak tekrar gelmelerini sağlayın.\n"
    }

    for segment in top_segments:
        print("-" * 40)
        if segment in strategies:
            print(strategies[segment])
        else:
            print(f"📌 [SEGMENT: {segment.upper()}]")
            print("   Action: Standart iletişim planını sürdürün.")
            print("   Taktik: Genel kampanya dönemlerinde hatırlatma yapın.\n")

    print("="*65)
    # Sadece teknik sürecin bittiğini belirten net ifade:
    print("[INFO] RFM Analizi tamamlandı. Aksiyon planı oluşturuldu.")

# Fonksiyonu Çalıştır
recommend_actions(final_rfm)


STRATEJİK AKSİYON PLANI (YÖNETİCİ ÖZETİ)
----------------------------------------
💤 [SEGMENT: UYKUDAKİLER]
   Durum: Uzun süredir gelmediler ve alışveriş sıklıkları düşüktü.
   Action: Maliyet Odaklı Geri Kazanım.
   Taktik: Sadece e-posta kanalıyla, 'Seni Özledik' temalı agresif bir indirim kuponu iletin.

----------------------------------------
🔄 [SEGMENT: SADIK MÜŞTERİLER]
   Durum: Düzenli geliyorlar ancak harcamaları arttırılabilir.
   Action: Sepet Ortalamasını Yükseltme (Upsell/Cross-sell).
   Taktik: İlgi alanlarına göre yan ürün önerileri yapın. Kargo bedava limitini hatırlatın.

----------------------------------------
✨ [SEGMENT: ŞAMPİYONLAR]
   Durum: En son, en sık ve en çok alışveriş yapanlar.
   Action: 'Marka Elçisi' programına dahil edin.
   Taktik: Yeni ürünleri ilk onlara sunun (Erken Erişim). Özel %5 teşekkür indirimi tanımlayın.

[INFO] RFM Analizi tamamlandı. Aksiyon planı oluşturuldu.
